# Тестирование Spell Checker

В этом ноутбуке будет произведено базовое тестировние написанной модели.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import json
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)

from IPython.display import display
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

In [4]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

In [5]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'r') as inf:
    handcode_table = json.load(inf)
candidate_generator = CandidateGenerator(
    words=vocab.keys(), handcode_table=handcode_table, max_distance=1
)

2021-01-24 20:11:12.914 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/mrgeekman/Documents/MIPT/НИР/Repo/data/external/russian_words/russian_words_vocab.dict]


In [6]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
position_selector = KenlmPositionSelector(model_left_right, model_right_left)

In [7]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
scorer_basis = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = np.mean
candidate_scorer = BertCandidateScorer(scorer_basis, agg_subtoken_func)

Some weights of the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a d

In [8]:
stopping_criteria = MarginStoppingCriteria(np.log(2.5))

In [9]:
# количество кандидатов, отбираемых position selector
num_selected_candidates = 16
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    stopping_criteria,
    tokenizer,
    detokenizer,
    num_selected_candidates,
    max_it
)

## Тестирование

In [10]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [11]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) // batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [12]:
!mkdir -p ../data/processed/results/

In [13]:
with open(os.path.join(DATA_PATH, 'processed', 'results', 'test.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [14]:
!python ../src/evaluation/spell_ru_eval/evaluate.py ../data/external/spell_ru_eval/train_source.txt ../data/external/spell_ru_eval/train_corrected.txt ../data/processed/results/test.txt | tail -n 2

Precision=62.16 Recall=65.26 FMeasure=63.67
1127 1813 1727


Как видим, имеем
* True Positive: $1127$
* Внесенных исправлений: $1813$ 
* Требуемых исправления: $1727$ 
* Precision: $62.16$
* Recall: $65.26$
* FMeasure: $63.67$

## Выводы

1. Пока результат выглядит не очень хорошо, особенно смущает precision. Казалось, что BERT сможет более качественно выполнять исправления. Возможно, эта проблема решится если использовать какую-то модель ошибок и скоры из position selector для окончательного выбора кандидата.
2. Пробовал заменить модель в position selector на 3-граммную модель, которая обучалась без текстов из соц. сетей -- результат примерно такой же.
3. Пробовал заменить модель в position selector на 4-граммную модель -- результат примерно такой же.
4. Пробовал увеличить максимальное расстояние Дамера-Левенштейна до двух -- результат стал сильно хуже.
5. Пробовал изменить аггрегирующую функцию по WordPiece-токенам в candidate scorer -- результат стал немного хуже.